# 라이브러리

In [ ]:
# 파이썬 표준 라이브러리
from typing import Generator

import os
import json
from collections import defaultdict

# 파이썬 서드파티 라이브러리
from langchain_community.callbacks.openai_info import OpenAICallbackHandler

from dotenv import load_dotenv
from langchain_core.globals import set_llm_cache
from langchain_core.caches import InMemoryCache
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, FewShotPromptTemplate, PromptTemplate
from langchain_core.example_selectors.semantic_similarity import MaxMarginalRelevanceExampleSelector
from langchain_chroma import Chroma

from langchain_community.callbacks import get_openai_callback

# 전역 변수 및 환경 설정

In [2]:
load_dotenv()

True

In [3]:
openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key
folder_path = r'C:\Users\RMARKET\Desktop\STFO\STFO'
json_path = folder_path + r'\LLM_Info\llm_info.json'

In [6]:
if os.path.exists(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        llm_info = json.load(f)
else:
    llm_info = defaultdict(list)

# 함수

In [7]:
# LLM_Info에 전체 비용, 전체 프롬프트 토큰 갯수, 전체 답변 토큰 갯수, 전체 토큰 갯수를 추가하는 함수
def add_info(llm_info: dict[str, float, int],
             callback: Generator[OpenAICallbackHandler, None, None]) -> None:
    """llm_info에 비용, 토큰 갯수들을 추가하는 함수

    Args:
        llm_info: LLM 비용, 토큰 갯수 등의 정보를 가지고 있는 Dictionary
        callback: callback
    
    Returns:
        None
    
    """

    total_cost = callback.total_cost
    total_prompt_tokens = callback.prompt_tokens
    total_completion_tokens = callback.completion_tokens
    total_tokens = callback.total_tokens
    llm_info['Total_Cost(USD)'].append(total_cost)
    llm_info['Total_Prompt_Tokens'].append(total_prompt_tokens)
    llm_info['Total_Completion_Tokens'].append(total_completion_tokens)
    llm_info['Total_Tokens'].append(total_tokens)
    return None

# Main

In [9]:
# 인메모리 캐시를 사용
set_llm_cache(InMemoryCache())

chat = ChatOpenAI(model="gpt-4o-mini", api_key=openai_api_key, temperature=0.2)
messages = [
                SystemMessage(content="당신은 여행사 직원입니다. 사용자에게 여행 일정을 제공할 수 있습니다."),
                HumanMessage(content="서울에서 관광객이 많이 찾는 3대 명소는 어디예요?"),
            ]

with get_openai_callback() as callback:
    aiMessage = chat.invoke(messages)
    print(aiMessage.content)
    messages.append(aiMessage)
    add_info(llm_info=llm_info, callback=callback)
    
    messages.append(HumanMessage(content="방금 당신이 알려준 3대 명소들에 대해 다시 알려주세요."))
    aiMessage = chat.invoke(messages)
    add_info(llm_info=llm_info, callback=callback)
    print(aiMessage.content)

서울에서 관광객이 많이 찾는 3대 명소는 다음과 같습니다:

1. **경복궁**: 조선 왕조의 주요 궁궐로, 아름다운 전통 건축과 정원이 인상적입니다. 경복궁 내에는 국립민속박물관과 국립고궁박물관도 있어 한국의 역사와 문화를 깊이 있게 체험할 수 있습니다.

2. **N서울타워 (남산타워)**: 서울의 상징적인 랜드마크로, 전망대에서 서울 시내 전경을 한눈에 볼 수 있습니다. 특히 야경이 아름다워 많은 관광객들이 방문합니다. 타워 주변의 남산공원도 산책하기 좋은 장소입니다.

3. **명동**: 쇼핑과 먹거리가 풍부한 명동은 외국인 관광객들에게 인기 있는 지역입니다. 다양한 브랜드 매장과 길거리 음식, 카페들이 있어 활기찬 분위기를 즐길 수 있습니다.

이 외에도 서울에는 많은 명소가 있으니, 여행 계획에 참고하시기 바랍니다!
물론입니다! 서울에서 관광객이 많이 찾는 3대 명소는 다음과 같습니다:

1. **경복궁**: 
   - 조선 왕조의 주요 궁궐로, 한국 전통 건축의 아름다움을 느낄 수 있는 곳입니다. 
   - 궁궐 내부에는 국립민속박물관과 국립고궁박물관이 있어 한국의 역사와 문화를 배울 수 있는 기회를 제공합니다.
   - 매일 정오에 열리는 수문장 교대식도 인기 있는 볼거리입니다.

2. **N서울타워 (남산타워)**: 
   - 서울의 상징적인 랜드마크로, 남산 위에 위치해 있어 서울 전경을 한눈에 볼 수 있는 전망대가 있습니다.
   - 특히 야경이 아름다워 많은 관광객들이 저녁에 방문합니다.
   - 타워 주변의 남산공원은 산책하기 좋은 장소로, 자연을 즐기며 여유로운 시간을 보낼 수 있습니다.

3. **명동**: 
   - 쇼핑과 먹거리가 풍부한 서울의 대표적인 상업 지역입니다.
   - 다양한 브랜드 매장과 화장품 가게, 길거리 음식, 카페들이 있어 활기찬 분위기를 즐길 수 있습니다.
   - 특히 외국인 관광객들에게 인기 있는 지역으로, 다양한 한국 음식을 맛볼 수 있는 기회도 많습니다.

이 명소들은 서울의 매력을 잘 보여주는 곳들이

## LLM 비용, 프롬프트 토큰 갯수, 답변 토큰 갯수, 전체 토큰 갯수

In [10]:
with open(json_path, 'w') as f:
    json.dump(llm_info, f, ensure_ascii=False)

In [ ]:
examples = [
    {
        "input": "Please summarize the weather news.\n",
        "summary": "Today's weather: Sunny skies, mild temperatures,"
        " and a gentle breeze. Enjoy the pleasant conditions throughout the day!",
    },
    {
        "input": "Please summarize the economy news.\n",
        "summary": "Global stocks rise on positive economic data;"
        "inflation concerns persist. Tech sector outperforms; central banks closely monitor.",
    },
    {
        "input": "Please summarize retail news.\n",
        "summary": "Major retailer announces record-breaking sales during holiday shopping season",
    },
    {
        "input": "What is stock market trend?\n",
        "summary": "Investor optimism grows amid easing global trade tensions",
    },
    {
        "input": "Typhoon related news.\n",
        "summary": "IAirports and schools close ahead of approaching typhoon threat",
    },
]

example_prompt = PromptTemplate(
    template="Input:{input} Summary:{summary}", input_variables=["input", "summary"]
)
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    Chroma,
    k=2,
)
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="input: {input}\nSummary:",
    prefix="",
    input_variables=["input"],
)

output = dynamic_prompt.format(
    input="I want to know the economy trends and weather this week."
)
print(output)

Number of requested results 20 is greater than number of elements in index 5, updating n_results = 5


Input:Please summarize the weather news.
 Summary:Today's weather: Sunny skies, mild temperatures, and a gentle breeze. Enjoy the pleasant conditions throughout the day!

Input:What is stock market trend?
 Summary:Investor optimism grows amid easing global trade tensions

input: I want to know the economy trends and weather this week.
Summary:


In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field  # Pydantic V2

model = ChatOpenAI(model="gpt-4o-mini", api_key=openai_api_key)

email = """
## Subject: Invitation to Generative AI Product Showcase

Dear [Recipient Name],

I am Terry, CTO of Exxon. I am pleased to invite you to a Generative AI Product Showcase at the Exxon office in Mountain View on December 1, 2025.

At this event, we will discuss Exxon's latest Generative AI technologies and how they can be used to develop new products and services. Specifically, you will learn about the following topics:

* Overview of Generative AI and its key features
* Exxon's Generative AI products and services
* Case studies of new products and services developed using Generative AI

The event will be attended by Terry, CTO of Exxon, and leaders from the Generative AI team. By attending, you will gain up-to-date information on Generative AI technologies and the opportunity to collaborate with Exxon to develop new products and services.

**Event Information:**

* Date: December 1, 2025
* Time: 10:00 AM
* Location: Exxon Office (Mountain View)

**RSVP:**

Please RSVP by email (terrycho@exon.example) or phone (609-123-1234) by November 20, 2025.

We look forward to seeing you there.

**Thank you.**

**Terry**

**CTO, Exxon**

**Contact:**

* Email: terrycho@exon.example
* Phone: 609-123-1234"""


class EmailParser(BaseModel):
    sender_name: str = Field(description="Person who send email")
    sender_title: str = Field(description="Job title of the email sender")
    sender_contact_email: str = Field(description="Email address of the email sender")
    sender_contact_phone: str = Field(description="Phone number of the email sender")
    email_type: str = Field(
        description="Type of email,it can be personal email, business email, spam, newletter,notificaion"
    )
    summary: str = Field(description="Short description of the email in 50 words")
    date: str = Field(
        description="If this email is meeting inviation, this is meeting date and time"
    )


parser = JsonOutputParser(pydantic_object=EmailParser)
prompt_template = PromptTemplate(
    template="Parse the email .\n{format_instructions}\n{email}\n",
    input_variables=["email"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt = prompt_template.format(email=email)
output = model.invoke(prompt)
output_text = parser.invoke(output)
print(output_text)

{'sender_name': 'Terry', 'sender_title': 'CTO', 'sender_contact_email': 'terrycho@exon.example', 'sender_contact_phone': '609-123-1234', 'email_type': 'notification', 'summary': "Invitation to a Generative AI Product Showcase at the Exxon office in Mountain View on December 1, 2025, discussing Exxon's latest technologies.", 'date': 'December 1, 2025'}


In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

customer_review = """\
I ordered spaghetti delivery, and it tasted good. \
However, the quantity is small compared to the price. \
It took too long for it to be delivered after ordering, so it was cold.
"""

review_template = """\
For the following text, extract the following information:

affordability : if customer feels that the price of the food is high output is expensive. \
if customer feels that the price of the food is affordable, or customer doesn't mention the price,\
output if affordable.

quality : If the customer is satisfied with the food quality, output 3, \
if there is no mention or the food quality is average, output 2,\
and if the customer says it is bad, output 1.

delivery_time : If the delivery time is fast, mark it as 3, \
if it is average, mark it as 2, and if it is late, mark it as 1.

Format the output as JSON with the following keys:
affordability
quality
delivery_time

text: {text}
"""

chat = ChatOpenAI(model="gpt-4o-mini", api_key=openai_api_key)
prompt_template = ChatPromptTemplate.from_template(review_template)
messages = prompt_template.format_messages(text=customer_review)
response = chat.invoke(messages)
print(response.content)

```json
{
  "affordability": "expensive",
  "quality": 3,
  "delivery_time": 1
}
```


In [22]:
from langchain.memory import ConversationBufferWindowMemory
from langchain_openai import OpenAI

model = OpenAI(openai_api_key=openai_api_key)
# k는 저장할 대화의 수를 의미
memory = ConversationBufferWindowMemory(k=2, memory_key='chat_history', return_messages=True)
memory.clear()
memory.save_context({'input': "Hello chabot!"}, {'output': 'Hello. How can I help you?'})
memory.save_context({'input': 'My name is Terry'}, {'output': "Nice to meet you Terry"})
memory.save_context({"input": "Where is Seoul?"}, {"output": "Seoul is in Korea"})

memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='My name is Terry', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you Terry', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Where is Seoul?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Seoul is in Korea', additional_kwargs={}, response_metadata={})]}

In [23]:
with open(r'C:\Users\RMARKET\Desktop\STFO\STFO\Data\Hankyung_Data.json', 'r', encoding='utf-8') as f:
    data_json = json.load(f)

In [77]:
from langchain_core.documents import Document

docs = [
        Document(page_content=news_info['news_content'], metadata={"source": news_info['news_url'], "title": news_info['news_title'],
                                                                   "date": news_info['news_first_upload_time'], "website": news_info['news_website']})
            for news_info in data_json]

In [78]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [79]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
texts = text_splitter.split_documents(docs)

In [81]:
embeddings = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(docs,
                                   embedding = embeddings
                                  )
vectorstore

In [134]:
query = '대한민국'
result = vectorstore.similarity_search(query)

In [135]:
len(result)

4

In [136]:
result[0].page_content

'거액의 가상자산(암호화폐) 보유 논란으로 더불어민주당에서 탈당한 김남국 무소속 의원이 개인 페이스북을 통해 "22대 총선에 불출마하겠다"고 22일 밝혔다.\n아주경제의 보도에 따르면 김 의원은 "제 문제로 심려를 끼쳐드려 다시 한번 고개 숙여 사과드린다. 제 징계안에 대해 현재 국회 윤리위원회에서 심의 중"이라며 이같이 언급했다.\n그는 "청년정치인에게 국회에서 일할 기회를 주신 안산 단원을 유권자 여러분께 은혜를 갚고 성과로 보답하고자 했으나, 실망을 안겨드려 마음이 무겁다"며 "제 간절한 바람이 있다면, 저를 믿고 응원해 준 안산시민을 위해 임기 끝까지 책임을 다하는 것 뿐이다. 남은 임기 동안 하루를 쪼개고 쪼개어 안산시민 여러분과 함께하겠다"고 말했다.'

In [137]:
mmr_docs = vectorstore.max_marginal_relevance_search(query, k=4, fetch_k=10)
print(len(mmr_docs))
print(mmr_docs[0].page_content)

4
거액의 가상자산(암호화폐) 보유 논란으로 더불어민주당에서 탈당한 김남국 무소속 의원이 개인 페이스북을 통해 "22대 총선에 불출마하겠다"고 22일 밝혔다.
아주경제의 보도에 따르면 김 의원은 "제 문제로 심려를 끼쳐드려 다시 한번 고개 숙여 사과드린다. 제 징계안에 대해 현재 국회 윤리위원회에서 심의 중"이라며 이같이 언급했다.
그는 "청년정치인에게 국회에서 일할 기회를 주신 안산 단원을 유권자 여러분께 은혜를 갚고 성과로 보답하고자 했으나, 실망을 안겨드려 마음이 무겁다"며 "제 간절한 바람이 있다면, 저를 믿고 응원해 준 안산시민을 위해 임기 끝까지 책임을 다하는 것 뿐이다. 남은 임기 동안 하루를 쪼개고 쪼개어 안산시민 여러분과 함께하겠다"고 말했다.
